[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PTB-MR/mrpro/blob/main/examples/notebooks/iterative_sense_reconstruction_with_regularization.ipynb)

In [ ]:
import importlib

if not importlib.util.find_spec('mrpro'):
    %pip install mrpro[notebook]

# Regularized Iterative SENSE Reconstruction of 2D golden angle radial data
Here we use the `mrpro.algorithms.reconstruction.RegularizedIterativeSENSEReconstruction` class to reconstruct
undersampled images from 2D radial data.

In [ ]:
# Download raw data from Zenodo
import tempfile
from pathlib import Path

import zenodo_get

dataset = '14617082'

tmp = tempfile.TemporaryDirectory()  # RAII, automatically cleaned up
data_folder = Path(tmp.name)
zenodo_get.zenodo_get([dataset, '-r', 5, '-o', data_folder])  # r: retries

### Image reconstruction
We use the `~mrpro.algorithms.reconstruction.RegularizedIterativeSENSEReconstruction` class to reconstruct images from 2D radial data.
It solves the following reconstruction problem:

Let's assume we have obtained the k-space data $y$ from an image $x$ with an acquisition model (Fourier transforms,
coil sensitivity maps...) $A$ then we can formulate the forward problem as:

$ y = Ax + n $

where $n$ describes complex Gaussian noise. The image $x$ can be obtained by minimizing the functionl $F$

$ F(x) = ||W^{\frac{1}{2}}(Ax - y)||_2^2 $

where $W^\frac{1}{2}$ is the square root of the density compensation function (which corresponds to a diagonal
operator). Because this is an ill-posed problem, we can add a regularization term to stabilize the problem and obtain
a solution with certain properties:

$ F(x) = ||W^{\frac{1}{2}}(Ax - y)||_2^2 + l||Bx - x_{reg}||_2^2$

where $l$ is the strength of the regularization, $B$ is a linear operator and $x_{reg}$ is a regularization image.
With this functional $F$ we obtain a solution which is close to $x_{reg}$ and to the acquired data $y$.

Setting the derivative of the functional $F$ to zero and rearranging yields

$ (A^H W A + l B) x = A^H W y + l x_{reg}$

which is a linear system $Hx = b$ that needs to be solved for $x$.

One important question of course is, what to use for $x_{reg}$. For dynamic images (e.g. cine MRI) low-resolution
dynamic images or high-quality static images have been proposed. In recent years, also the output of neural-networks
has been used as an image regulariser.

In this example we are going to use a high-quality image to regularize the reconstruction of an undersampled image.
Both images are obtained from the same data acquisition (one using all the acquired data ($x_{reg}$) and one using
only parts of it ($x$)). This of course is an unrealistic case but it will allow us to study the effect of the
regularization.

##### Read-in the raw data

In [ ]:
# Download raw data from Zenodo
import tempfile
from pathlib import Path

import mrpro
import torch
import zenodo_get

dataset = '14617082'

tmp = tempfile.TemporaryDirectory()  # RAII, automatically cleaned up
data_folder = Path(tmp.name)
zenodo_get.zenodo_get([dataset, '-r', 5, '-o', data_folder])  # r: retries

### Reading of both fully sampled and undersampled data
This will use the trajectory that is stored in the ISMRMRD file.

In [ ]:
# Read the raw data and the trajectory from ISMRMRD file

kdata_fullysampled = mrpro.data.KData.from_file(
    data_folder / 'radial2D_402spokes_golden_angle_with_traj.h5',
    mrpro.data.traj_calculators.KTrajectoryIsmrmrd(),
)
kdata_undersampled = mrpro.data.KData.from_file(
    data_folder / 'radial2D_24spokes_golden_angle_with_traj.h5',
    mrpro.data.traj_calculators.KTrajectoryIsmrmrd(),
)

##### Image $x_{reg}$ from fully sampled data
We first reconstruct the fully sampled image to use it as a regularization image.
In a real-world scenario, we would not have this image and would have to use a low-resolution image as a prior, or use
a neural network to estimate the regularization image.

In [ ]:
# Estimate coil maps. Here we use the fully sampled data to estimate the coil sensitivity maps.
# In a real-world scenario, we would either a calibration scan (e.g. a separate fully sampled scan) to estimate the coil
# sensitivity maps or use ESPIRiT or similar methods to estimate the coil sensitivity maps from the undersampled data.
direct_reconstruction = mrpro.algorithms.reconstruction.DirectReconstruction(kdata_fullysampled)
csm = direct_reconstruction.csm
assert csm is not None

# unregularized iterative SENSE reconstruction of the fully sampled data
iterative_sense_reconstruction = mrpro.algorithms.reconstruction.IterativeSENSEReconstruction(
    kdata_fullysampled, csm=csm, n_iterations=3
)
img_iterative_sense = iterative_sense_reconstruction(kdata_fullysampled)

##### Image $x$ from undersampled data

In [ ]:
# unregularized iterative SENSE reconstruction of the undersampled data
iterative_sense_reconstruction = mrpro.algorithms.reconstruction.IterativeSENSEReconstruction(
    kdata_undersampled, csm=csm, n_iterations=6
)
img_us_iterative_sense = iterative_sense_reconstruction(kdata_undersampled)

In [ ]:
# Regularized iterativ SENSE reconstruction of the undersampled data

regularized_iterative_sense_reconstruction = mrpro.algorithms.reconstruction.RegularizedIterativeSENSEReconstruction(
    kdata_undersampled,
    csm=csm,
    n_iterations=6,
    regularization_data=img_iterative_sense.data,
    regularization_weight=1.0,
)
img_us_regularized_iterative_sense = regularized_iterative_sense_reconstruction(kdata_undersampled)

##### Display the results

In [ ]:
import matplotlib.pyplot as plt


def show_images(*images: torch.Tensor, titles: list[str] | None = None) -> None:
    """Plot images."""
    n_images = len(images)
    _, axes = plt.subplots(1, n_images, squeeze=False, figsize=(n_images * 3, 3))
    for i in range(n_images):
        axes[0][i].imshow(images[i], cmap='gray')
        axes[0][i].axis('off')
        if titles:
            axes[0][i].set_title(titles[i])
    plt.show()

In [ ]:

# vis_im = [img_iterative_sense.rss(), img_us_iterative_sense.rss(), img_us_regularized_iterative_sense.rss()]
# vis_title = ['Fully sampled', 'Iterative SENSE R=20', 'Regularized Iterative SENSE R=20']
# fig, ax = plt.subplots(1, 3, squeeze=False, figsize=(12, 4))
# for ind in range(3):
#     ax[0, ind].imshow(vis_im[ind][0, 0, ...])
#     ax[0, ind].set_title(vis_title[ind])
show_images(
    img_iterative_sense.rss()[0, 0],
    img_us_iterative_sense.rss()[0, 0],
    img_us_regularized_iterative_sense.rss()[0, 0],
    titles=['Fully sampled', 'Iterative SENSE R=20', 'Regularized Iterative SENSE R=20'],
)

### Behind the scenes

##### Set-up the density compensation operator $W$ and acquisition model $A$

This is very similar to <project:iterative_sense_reconstruction.ipynb>. For more details, please refer to that notebook.

In [ ]:
dcf_operator = mrpro.data.DcfData.from_traj_voronoi(kdata_undersampled.traj).as_operator()
fourier_operator = mrpro.operators.FourierOp.from_kdata(kdata_undersampled)
csm_operator = csm.as_operator()
acquisition_operator = fourier_operator @ csm_operator

##### Calculate the right-hand-side of the linear system $b = A^H W y + l x_{reg}$

In [ ]:
regularization_weight = 1.0
regularization_image = img_iterative_sense.data

right_hand_side = (acquisition_operator.H @ dcf_operator(kdata_undersampled.data))[
    0
] + regularization_weight * regularization_image

##### Set-up the linear self-adjoint operator $H = A^H W A + l$

In [ ]:

operator = (
    acquisition_operator.H @ dcf_operator @ acquisition_operator + mrpro.operators.IdentityOp() * regularization_weight
)

##### Run conjugate gradient

In [ ]:
img_manual = mrpro.algorithms.optimizers.cg(
    operator, right_hand_side, initial_value=right_hand_side, max_iterations=8, tolerance=0.0
)

#####  Display the reconstructed image

In [ ]:
show_images(
    img_us_regularized_iterative_sense.rss()[0, 0],
    img_manual.abs()[0, 0],
    titles=['Regularized Iterative SENSE R=20', '"Manual" Regularized Iterative SENSE R=20'],
)

### Check for equal results
The two versions should result in the same image data.

In [ ]:
# If the assert statement did not raise an exception, the results are equal.
assert torch.allclose(img_us_regularized_iterative_sense.data, img_manual)

### Next steps

We are cheating here because we used the fully sampled image as a regularization. In real world applications
we would not have that. One option is to apply a low-pass filter to the undersampled k-space data to try to reduce the
streaking artifacts and use that as a regularization image. Try that and see if you can also improve the image quality
compared to the unregularised images.